In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchmetrics.classification import MulticlassAveragePrecision
import time
import os

# 기존 모듈들 import
from models.efficientnet_teacher import EfficientNetTeacher
from Dataset.data import OfficeHomeDataset

def evaluate_teacher_model(model, dataloader, device, dataset_name="Test"):
    """
    Teacher 모델 평가 함수
    """
    print(f"🎓 {dataset_name} 데이터셋에서 Teacher 모델 평가 중...")
    
    model.eval()
    start_time = time.time()
    
    # 손실 함수
    domain_criterion = nn.CrossEntropyLoss()
    class_criterion = nn.CrossEntropyLoss()
    
    # 통계 변수
    running_loss = 0.0
    running_domain_loss = 0.0
    running_class_loss = 0.0
    domain_correct = 0
    class_correct = 0
    total = 0
    
    # mAP 계산기 초기화
    domain_map = MulticlassAveragePrecision(num_classes=4, average='macro').to(device)
    class_map = MulticlassAveragePrecision(num_classes=65, average='macro').to(device)
    
    # 클래스별 정확도를 위한 리스트
    all_domain_preds = []
    all_domain_labels = []
    all_class_preds = []
    all_class_labels = []
    
    with torch.no_grad():
        for batch_idx, (inputs, domain_labels, class_labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            domain_labels = domain_labels.to(device)
            class_labels = class_labels.to(device)
            
            # 순전파
            domain_outputs, class_outputs = model(inputs)
            
            # 손실 계산
            domain_loss = domain_criterion(domain_outputs, domain_labels)
            class_loss = class_criterion(class_outputs, class_labels)
            loss = 0.5 * domain_loss + 0.5 * class_loss  # Teacher 훈련 시와 동일한 가중치
            
            running_loss += loss.item()
            running_domain_loss += domain_loss.item()
            running_class_loss += class_loss.item()
            
            # 정확도 계산
            _, domain_preds = domain_outputs.max(1)
            domain_correct += domain_preds.eq(domain_labels).sum().item()
            
            _, class_preds = class_outputs.max(1)
            class_correct += class_preds.eq(class_labels).sum().item()
            
            total += inputs.size(0)
            
            # mAP 업데이트
            domain_map.update(domain_outputs, domain_labels)
            class_map.update(class_outputs, class_labels)
            
            # 예측값 저장 (추가 분석용)
            all_domain_preds.extend(domain_preds.cpu().numpy())
            all_domain_labels.extend(domain_labels.cpu().numpy())
            all_class_preds.extend(class_preds.cpu().numpy())
            all_class_labels.extend(class_labels.cpu().numpy())
            
            # 진행 상황 출력
            if (batch_idx + 1) % 20 == 0:
                print(f'Batch [{batch_idx+1}/{len(dataloader)}] processed...')
    
    # 최종 통계 계산
    avg_loss = running_loss / len(dataloader)
    avg_domain_loss = running_domain_loss / len(dataloader)
    avg_class_loss = running_class_loss / len(dataloader)
    domain_accuracy = 100.0 * domain_correct / total
    class_accuracy = 100.0 * class_correct / total
    
    # mAP 계산
    domain_map_value = domain_map.compute().item()
    class_map_value = class_map.compute().item()
    
    eval_time = time.time() - start_time
    
    # 결과 출력
    print(f"\n{'='*60}")
    print(f"🎓 EfficientNet Teacher 모델 성능 평가 결과")
    print(f"{'='*60}")
    print(f"데이터셋: {dataset_name}")
    print(f"총 샘플 수: {total:,}")
    print(f"평가 시간: {eval_time:.2f}초")
    print(f"\n📊 손실 (Loss)")
    print(f"  전체 손실: {avg_loss:.4f}")
    print(f"  도메인 손실: {avg_domain_loss:.4f}")
    print(f"  클래스 손실: {avg_class_loss:.4f}")
    print(f"\n🎯 정확도 (Accuracy)")
    print(f"  도메인 정확도: {domain_accuracy:.2f}%")
    print(f"  클래스 정확도: {class_accuracy:.2f}%")
    print(f"  평균 정확도: {(domain_accuracy + class_accuracy)/2:.2f}%")
    print(f"\n🏆 mAP (Mean Average Precision)")
    print(f"  도메인 mAP: {domain_map_value:.4f}")
    print(f"  클래스 mAP: {class_map_value:.4f}")
    print(f"  평균 mAP: {(domain_map_value + class_map_value)/2:.4f}")
    print(f"{'='*60}")
    
    return {
        'total_loss': avg_loss,
        'domain_loss': avg_domain_loss,
        'class_loss': avg_class_loss,
        'domain_accuracy': domain_accuracy,
        'class_accuracy': class_accuracy,
        'domain_map': domain_map_value,
        'class_map': class_map_value,
        'eval_time': eval_time,
        'total_samples': total
    }

def main():
    # 디바이스 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # 모델 생성
    teacher = EfficientNetTeacher(num_domains=4, num_classes=65).to(device)
    
    # 체크포인트 로드
    checkpoint_path = 'efficientnet_teacher_best.pth'
    
    if not os.path.exists(checkpoint_path):
        print(f"❌ 에러: {checkpoint_path} 파일을 찾을 수 없습니다!")
        print("다음 중 하나를 확인해주세요:")
        print("1. 파일이 현재 디렉토리에 있는지 확인")
        print("2. 파일 경로가 올바른지 확인") 
        print("3. Teacher 모델이 훈련되어 저장되었는지 확인")
        return
    
    print(f"📁 체크포인트 로드 중: {checkpoint_path}")
    try:
        # GPU에서 훈련된 모델을 CPU에서도 로드할 수 있도록
        teacher.load_state_dict(torch.load(checkpoint_path, map_location=device))
        print("✅ 모델 가중치 로드 완료!")
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return
    
    # 데이터 변환 설정 (훈련 시와 동일)
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # 테스트 데이터셋 로드
    print("📂 테스트 데이터셋 로드 중...")
    testset = OfficeHomeDataset(root_dir='./Dataset/test', transform=transform_test)
    testloader = DataLoader(
        testset, 
        batch_size=128,  # 평가용이므로 큰 배치 사이즈 사용
        shuffle=False, 
        pin_memory=True, 
        num_workers=8
    )
    
    print(f"테스트 세트 크기: {len(testset)}")
    
    # 훈련 데이터셋도 평가 (선택적)
    print("📂 훈련 데이터셋 로드 중...")
    trainset = OfficeHomeDataset(root_dir='./Dataset/train', transform=transform_test)
    trainloader = DataLoader(
        trainset, 
        batch_size=128, 
        shuffle=False, 
        pin_memory=True, 
        num_workers=8
    )
    
    print(f"훈련 세트 크기: {len(trainset)}")
    
    # GPU 병렬 처리 (필요시)
    if torch.cuda.device_count() > 1:
        print(f"🚀 {torch.cuda.device_count()}개 GPU 사용")
        teacher = nn.DataParallel(teacher)
    
    # 테스트 데이터셋 평가
    test_results = evaluate_teacher_model(teacher, testloader, device, "Test")
    
    # 훈련 데이터셋 평가 (오버피팅 확인용)
    train_results = evaluate_teacher_model(teacher, trainloader, device, "Train")
    
    # 오버피팅 분석
    print(f"\n🔍 오버피팅 분석")
    print(f"{'='*40}")
    domain_gap = train_results['domain_accuracy'] - test_results['domain_accuracy']
    class_gap = train_results['class_accuracy'] - test_results['class_accuracy']
    domain_map_gap = train_results['domain_map'] - test_results['domain_map']
    class_map_gap = train_results['class_map'] - test_results['class_map']
    
    print(f"도메인 정확도 차이: {domain_gap:.2f}% (Train - Test)")
    print(f"클래스 정확도 차이: {class_gap:.2f}% (Train - Test)")
    print(f"도메인 mAP 차이: {domain_map_gap:.4f} (Train - Test)")
    print(f"클래스 mAP 차이: {class_map_gap:.4f} (Train - Test)")
    
    if domain_gap > 5 or class_gap > 5:
        print("⚠️  오버피팅 가능성이 있습니다.")
    else:
        print("✅ 일반화 성능이 양호합니다.")

if __name__ == "__main__":
    main()

Using device: cuda


/home/guswls/.local/lib/python3.11/site-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnet_b2_ns to current tf_efficientnet_b2.ns_jft_in1k.
  model = create_fn(


📁 체크포인트 로드 중: efficientnet_teacher_best.pth
✅ 모델 가중치 로드 완료!
📂 테스트 데이터셋 로드 중...
총 3213 이미지, 65 클래스, 4 도메인을 로드했습니다.
테스트 세트 크기: 3213
📂 훈련 데이터셋 로드 중...
총 11113 이미지, 65 클래스, 4 도메인을 로드했습니다.
훈련 세트 크기: 11113
🚀 2개 GPU 사용
🎓 Test 데이터셋에서 Teacher 모델 평가 중...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a CuDNNError: cuDNN error: CUDNN_STATUS_BAD_PARAM
Exception raised from run_conv_plan at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:374 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7f6e6008a897 in /usr/local/lib/python3.11/dist-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0xe1c861 (0x7f6ded5ef861 in /usr/local/lib/python3.11/dist-packages/torch/lib/libtorch_cuda.so)
frame #2: <unknown function> + 0x1095d83 (0x7f6ded868d83 in /usr/local/lib/python3.11/dist-packages/torch/

Batch [20/26] processed...

🎓 EfficientNet Teacher 모델 성능 평가 결과
데이터셋: Test
총 샘플 수: 3,213
평가 시간: 72.95초

📊 손실 (Loss)
  전체 손실: 0.9865
  도메인 손실: 1.2026
  클래스 손실: 0.7705

🎯 정확도 (Accuracy)
  도메인 정확도: 83.38%
  클래스 정확도: 85.03%
  평균 정확도: 84.20%

🏆 mAP (Mean Average Precision)
  도메인 mAP: 0.8867
  클래스 mAP: 0.9011
  평균 mAP: 0.8939
🎓 Train 데이터셋에서 Teacher 모델 평가 중...
Batch [20/87] processed...
Batch [40/87] processed...
Batch [60/87] processed...
Batch [80/87] processed...

🎓 EfficientNet Teacher 모델 성능 평가 결과
데이터셋: Train
총 샘플 수: 11,113
평가 시간: 233.00초

📊 손실 (Loss)
  전체 손실: 0.0070
  도메인 손실: 0.0021
  클래스 손실: 0.0119

🎯 정확도 (Accuracy)
  도메인 정확도: 99.93%
  클래스 정확도: 99.40%
  평균 정확도: 99.66%

🏆 mAP (Mean Average Precision)
  도메인 mAP: 1.0000
  클래스 mAP: 0.9998
  평균 mAP: 0.9999

🔍 오버피팅 분석
도메인 정확도 차이: 16.55% (Train - Test)
클래스 정확도 차이: 14.37% (Train - Test)
도메인 mAP 차이: 0.1133 (Train - Test)
클래스 mAP 차이: 0.0987 (Train - Test)
⚠️  오버피팅 가능성이 있습니다.
